In [ ]:
import os
import json
import math
import glob
import shutil
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import numpy as np
import cv2
from tqdm import tqdm
import inspect
import numpy as np
try:
    from cellpose import models
except Exception as e:
    models = None

In [ ]:
!pip install -q --no-deps --target /kaggle/working/cp_env "cellpose==4.0.7"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 10.5 MB/s eta 0:00:00


In [ ]:
# Required by Cellpose at import/run time
!pip install -q --no-deps --target /kaggle/working/cp_env fastremap fill-voids roifile imagecodecs

# Image IO / processing (no GUI)
!pip install -q --no-deps --target /kaggle/working/cp_env tifffile "scikit-image>=0.22" "opencv-python-headless<5"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 29.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.2/231.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 93.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 30.9 MB/s eta 0:00:00:00:0100:01


In [ ]:
import sys, os
sys.path.insert(0, "/kaggle/working/cp_env")

import numpy as np
print("NumPy path:", np.__file__)

from cellpose import models


NumPy path: /usr/local/lib/python3.11/dist-packages/numpy/__init__.py


In [ ]:
# Config
# ---------------------------
class Cfg:
    base_dir = os.environ.get("HPA_BASE", "/kaggle/input/hpa-single-cell-image-classification")
    #base_dir = os.environ.get("HPA_BASE", "/kaggle/working/hpa_subset")
    # Where to write outputs
    out_dir = os.environ.get("HPA_OUT", "/kaggle/working/hpa_segmentation")
    #out_dir = os.environ.get("HPA_OUT", "/kaggle/working/hpa_seg_test")
    splits = ["train"]

    # File pattern per channel
    patt_red = "*_red.png"
    patt_green = "*_green.png"
    patt_blue = "*_blue.png"
    patt_yellow = "*_yellow.png"

    # Cellpose
    model_type = "cellpose-sam"  # "cyto2" is robust for fluorescent cyto segmentation
    use_gpu = True
    diameter = None          # SAM is size-invariant; None/0 lets the model handle scale          # heuristic avg cell diameter in pixels; tune per dataset (30–80)
    flow_threshold = 0.5   # higher -> fewer, more confident masks
    cellprob_threshold = -0.8 # higher -> fewer masks; try -0.5..0.5
    min_area = 135         # remove tiny specks (in pixels)

    # Cropping/Resizing
    batch_size = 8
    crop_size = 224  # final square size per cell crop
    pad = 16         # pixels of context around bbox before resize
    save_format = "npy"  # "npz" (channels-first float32), or "png" (uint8 RGB w/ green as G and composite elsewhere)

    # INTENSITY NORMALIZATION
    # Per-channel percentile normalization (robust to outliers)
    p_lo, p_hi = 1, 99

    # QC filtering: require some signal
    min_green_mean = 1.0  # on 0–255 scale after normalization (set 0 to disable)

    # Visualization
    save_overlay_pngs = False



In [ ]:
# ---------------------------
# Utils
# ---------------------------

import inspect

def cp_eval_batch(cp, imgs: list, cfg: Cfg):
    """
    imgs: list of HxW float32 arrays (already normalized)
    Works across Cellpose v3/v4, returns list[masks], list[flows]
    """
    # sanitize inputs
    imgs = [im.astype(np.float32, copy=False) for im in imgs]

    wanted = dict(
        channels=[0,0],
        diameter=cfg.diameter,
        flow_threshold=cfg.flow_threshold,
        cellprob_threshold=cfg.cellprob_threshold,
        resample=True,
        augment=False,
        do_3D=False,
        batch_size=cfg.batch_size
    )
    sig = inspect.signature(cp.eval)
    kwargs = {k:v for k,v in wanted.items() if k in sig.parameters}

    res = cp.eval(imgs, **kwargs)

    if not isinstance(res, tuple):
        raise RuntimeError(f"Unexpected return type: {type(res)}")

    if len(res) == 4:
        masks, flows, _, _ = res
    elif len(res) == 3:
        masks, flows, _ = res
    elif len(res) == 2:
        masks, flows = res
    else:
        raise RuntimeError(f"Unexpected return length: {len(res)}")

    # Ensure lists
    if not isinstance(masks, (list, tuple)):
        masks = [masks]
    if not isinstance(flows, (list, tuple)):
        flows = [flows]

    # cast masks to int32
    masks = [np.asarray(m).astype(np.int32, copy=False) if m is not None else None for m in masks]
    return masks, flows


def ensure_dir(p: str):
    Path(p).mkdir(parents=True, exist_ok=True)


def read_hpa_four_channel(folder: str, stem: str) -> Optional[np.ndarray]:
    """Read 4 channels by filename stem (without _red/_green suffix).
    Returns array shape (4, H, W) in float32 0–255 or None if missing any channel.
    """
    paths = {
        'red':   os.path.join(folder, f"{stem}_red.png"),
        'green': os.path.join(folder, f"{stem}_green.png"),
        'blue':  os.path.join(folder, f"{stem}_blue.png"),
        'yellow':os.path.join(folder, f"{stem}_yellow.png"),
    }
    for k, p in paths.items():
        if not os.path.exists(p):
            return None
    imgs = []
    for k in ['red','green','blue','yellow']:
        im = cv2.imread(paths[k], cv2.IMREAD_UNCHANGED)
        if im is None:
            return None
        if im.ndim == 3:
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im = im.astype(np.float32)
        imgs.append(im)
    return np.stack(imgs, axis=0)  # (4, H, W)


def robust_percentile_norm(x: np.ndarray, p_lo=1, p_hi=99) -> np.ndarray:
    """Per-channel percentile normalization to 0–255 uint8 range."""
    x = x.copy()
    out = np.zeros_like(x, dtype=np.float32)
    for c in range(x.shape[0]):
        lo = np.percentile(x[c], p_lo)
        hi = np.percentile(x[c], p_hi)
        if hi <= lo:
            hi = lo + 1.0
        out[c] = np.clip((x[c] - lo) / (hi - lo), 0, 1) * 255.0
    return out.astype(np.float32)



In [ ]:
def apply_green_CLAHE(x_ch: np.ndarray, clip=1.0, tile=(8,8)) -> np.ndarray:
    clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=tile)
    return clahe.apply(x_ch.astype(np.uint8)).astype(np.float32)


def build_cyto_and_nuclei_inputs(x4: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Build grayscale inputs for Cellpose: cyto (max-projection of R/G/Y) and nuclei (Blue)."""
    red, green, blue, yellow = x4
    cyto = np.maximum.reduce([red, green, yellow])
    nuclei = blue
    return cyto, nuclei


def run_cellpose_cyto(cyto_img: np.ndarray, cfg: Cfg):
    if models is None:
        raise RuntimeError("cellpose not installed. !pip install cellpose[all] --quiet")
    cp = models.Cellpose(model_type=cfg.model_type, gpu=cfg.use_gpu)
    masks, flows, styles, diams = cp.eval(
        cyto_img,
        channels=[0, 0],          # grayscale-only input
        diameter=cfg.diameter,
        flow_threshold=cfg.flow_threshold,
        cellprob_threshold=cfg.cellprob_threshold,
        resample=True,
        augment=False,
        do_3D=False,
    )
    return masks.astype(np.int32), flows


def filter_instances(masks: np.ndarray, x4_norm: np.ndarray, cfg: Cfg) -> np.ndarray:
    """
    Keep plausible cells; drop obvious debris.
    - Signal metric: composite = max(R, G, Y)  (more robust than green-only)
    - Protect edge cells (partially visible) from area filtering
    """
    if masks.max() == 0:
        return masks

    H, W = masks.shape
    red, green, blue, yellow = x4_norm
    comp = np.maximum.reduce([red, green, yellow])  # composite cytoplasm

    # parameters
    min_area = max(0, int(getattr(cfg, "min_area", 50)))
    min_signal = float(getattr(cfg, "min_green_mean", 0.0))

    # precompute which labels touch border (edge cells)
    border = np.zeros_like(masks, dtype=bool)
    border[0, :] = border[-1, :] = border[:, 0] = border[:, -1] = True
    edge_labels = set(np.unique(masks[border])) - {0}

    keep = np.zeros(masks.max() + 1, dtype=bool)
    keep[0] = False

    for idx in range(1, masks.max() + 1):
        m = (masks == idx)
        area = int(m.sum())
        if area == 0:
            continue

        # mean composite intensity (0–255)
        sig = float(comp[m].mean())

        # relax filters for edge cells
        is_edge = idx in edge_labels
        area_ok = (area >= min_area) or is_edge
        sig_ok = (sig >= min_signal) or is_edge

        if area_ok and sig_ok:
            keep[idx] = True

    # remap labels to consecutive ints
    new_masks = np.zeros_like(masks, dtype=np.int32)
    new_id = 1
    for idx in range(1, masks.max() + 1):
        if keep[idx]:
            new_masks[masks == idx] = new_id
            new_id += 1
    return new_masks



In [ ]:
def bbox_from_mask(mask: np.ndarray) -> Tuple[int,int,int,int]:
    ys, xs = np.where(mask)
    y1, y2 = ys.min(), ys.max()
    x1, x2 = xs.min(), xs.max()
    return x1, y1, x2, y2


def crop_with_pad(x4: np.ndarray, inst_mask: np.ndarray, cfg: Cfg) -> np.ndarray:
    x1, y1, x2, y2 = bbox_from_mask(inst_mask)
    h, w = x4.shape[1:]
    x1 = max(0, x1 - cfg.pad)
    y1 = max(0, y1 - cfg.pad)
    x2 = min(w-1, x2 + cfg.pad)
    y2 = min(h-1, y2 + cfg.pad)
    crop = x4[:, y1:y2+1, x1:x2+1]
    # make square by padding
    ch, hh, ww = crop.shape
    side = max(hh, ww)
    pad_y = side - hh
    pad_x = side - ww
    crop_sq = np.pad(crop, ((0,0),(0,pad_y),(0,pad_x)), mode='constant')
    # resize to cfg.crop_size
    out = np.zeros((ch, cfg.crop_size, cfg.crop_size), dtype=np.float32)
    for c in range(ch):
        out[c] = cv2.resize(crop_sq[c], (cfg.crop_size, cfg.crop_size), interpolation=cv2.INTER_AREA)
    return out


def save_crop(out_path: str, crop4: np.ndarray, cfg: Cfg):
    ensure_dir(os.path.dirname(out_path))
    if cfg.save_format == "npy":
        if not out_path.endswith(".npy"):
            out_path = out_path + ".npy"
        # channels-first float32 (0–255)
        np.save(out_path, crop4.astype(np.float32), allow_pickle=False)
    elif cfg.save_format == "png":
        red, green, blue, yellow = crop4
        vis = np.stack([red, green, yellow], axis=-1).astype(np.uint8)
        cv2.imwrite(out_path, vis)
    else:
        raise ValueError("Unsupported save_format")


def colorize_masks(masks: np.ndarray) -> np.ndarray:
    rng = np.random.default_rng(42)
    H, W = masks.shape
    out = np.zeros((H, W, 3), dtype=np.uint8)
    for idx in range(1, masks.max()+1):
        color = rng.integers(64, 255, size=3, dtype=np.uint8)
        out[masks == idx] = color
    return out


def overlay_masks(x4_norm: np.ndarray, masks: np.ndarray, alpha=0.5) -> np.ndarray:
    # base RGB for viewing: R=red, G=green, B=blue
    base = np.stack([x4_norm[0], x4_norm[1], x4_norm[2]], axis=-1).astype(np.uint8)
    cm = colorize_masks(masks)
    over = cv2.addWeighted(base, 1.0, cm, alpha, 0)
    return over


In [ ]:
def process_folder(img_dir: str, out_dir: str, cfg: Cfg, stems_override: Optional[List[str]] = None) -> Dict[str, List[Dict]]:
    """
    Lightweight segmentation version:
      - No per-cell crop saving (prevents HDD overflow)
      - Saves 4-channel normalized image (.npy) per stem
      - Saves instance masks (.png)
      - Stores bbox + stats (area, centroid, mean green) in bags.json
    """
    import inspect, tempfile, json, time

    ensure_dir(out_dir)

    # --- discover stems (or use chunk slice) ---
    if stems_override is not None:
        stems = stems_override
        print(f"[INFO] Using provided subset: {len(stems)} stems")
    else:
        stems = sorted([
            os.path.basename(p).replace("_green.png", "")
            for p in glob.glob(os.path.join(img_dir, Cfg.patt_green))
        ])
    if len(stems) == 0:
        print(f"[WARN] No stems found in {img_dir}")
        return {}

    # --- init Cellpose ---
    if models is None:
        raise RuntimeError("Cellpose not installed. Run: !pip install -q 'cellpose[all]>=3.1'")

    try:
        cp = models.CellposeModel(gpu=cfg.use_gpu, model_type=cfg.model_type)  # v4.x
    except AttributeError:
        cp = models.Cellpose(model_type=cfg.model_type, gpu=cfg.use_gpu)       # ≤ v3.x

    # --- helper: version-agnostic eval ---
    def cp_eval_batch(imgs: list) -> Tuple[List[np.ndarray], List]:
        imgs = [im.astype(np.float32, copy=False) for im in imgs]
        wanted = dict(
            channels=[0, 0],
            diameter=cfg.diameter,
            flow_threshold=cfg.flow_threshold,
            cellprob_threshold=cfg.cellprob_threshold,
            resample=True,
            augment=False,
            do_3D=False,
            batch_size=getattr(cfg, "batch_size", 8),
        )
        sig = inspect.signature(cp.eval)
        kwargs = {k: v for k, v in wanted.items() if k in sig.parameters}
        res = cp.eval(imgs, **kwargs)
        if not isinstance(res, tuple):
            raise RuntimeError(f"Unexpected Cellpose eval return: {type(res)}")
        masks, flows = res[:2]
        if not isinstance(masks, (list, tuple)):
            masks = [masks]
        return [np.asarray(m).astype(np.int32) if m is not None else None for m in masks], flows

    # --- run segmentation ---
    bags: Dict[str, List[Dict]] = {}
    B = getattr(cfg, "batch_size", 8)
    batch_imgs, batch_stems, batch_x4s = [], [], []

    last_checkpoint = time.time()
    for i, stem in enumerate(tqdm(stems, desc=f"Segmenting {os.path.basename(img_dir)}")):
        x4_raw = read_hpa_four_channel(img_dir, stem)
        if x4_raw is None:
            continue

        # Normalize (disable CLAHE for speed)
        x4 = robust_percentile_norm(x4_raw, cfg.p_lo, cfg.p_hi)
        # x4[1] = apply_green_CLAHE(x4[1])  # optional

        cyto_img, _ = build_cyto_and_nuclei_inputs(x4)
        batch_imgs.append(cyto_img)
        batch_stems.append(stem)
        batch_x4s.append(x4)

        # process batch
        if len(batch_imgs) == B or stem == stems[-1]:
            masks_list, _ = cp_eval_batch(batch_imgs)

            for stem_b, x4_b, masks_b in zip(batch_stems, batch_x4s, masks_list):
                if masks_b is None:
                    masks_b = np.zeros_like(x4_b[0], dtype=np.int32)

                raw_n = int(masks_b.max())
                masks_b = filter_instances(masks_b, x4_b, cfg)
                kept_n = int(masks_b.max())
                print(f"[SEG] {stem_b}: raw={raw_n}, kept={kept_n}")

                # save normalized image (for MIL cropping later)
                img_path = os.path.join(out_dir, f"{stem_b}.npy")
                np.save(img_path, x4_b.astype(np.uint8), allow_pickle=False)

                # save optional overlay
                if cfg.save_overlay_pngs:
                    ov = overlay_masks(x4_b, masks_b)
                    ov_path = os.path.join(out_dir, f"{stem_b}_overlay.png")
                    cv2.imwrite(ov_path, ov)

                # collect cell metadata
                cell_entries: List[Dict] = []
                for idx in range(1, masks_b.max() + 1):
                    inst_mask = (masks_b == idx)
                    if inst_mask.sum() == 0:
                        continue
                    ys, xs = np.where(inst_mask)
                    y1, y2 = int(ys.min()), int(ys.max())
                    x1, x2 = int(xs.min()), int(xs.max())
                    cy, cx = float(ys.mean()), float(xs.mean())
                    area = int(inst_mask.sum())
                    gmean = float(x4_b[1][inst_mask].mean())

                    cell_entries.append({
                        "instance_id": idx,
                        "bbox_xyxy": [x1, y1, x2, y2],
                        "centroid_xy": [cx, cy],
                        "area": area,
                        "mean_green": gmean,
                    })

                bags[stem_b] = cell_entries

                # save mask safely
                mask_path = os.path.join(out_dir, f"{stem_b}_mask.png")
                tmp_fd, tmp_path = tempfile.mkstemp(prefix="tmpmask_", suffix=".png", dir=os.path.dirname(mask_path))
                os.close(tmp_fd)
                try:
                    ok = cv2.imwrite(tmp_path, masks_b.astype(np.uint16))
                    if ok:
                        os.replace(tmp_path, mask_path)
                except Exception as e:
                    print(f"[WARN] Failed to write mask for {stem_b}: {e}")
                    try:
                        os.remove(tmp_path)
                    except OSError:
                        pass

            batch_imgs.clear(); batch_stems.clear(); batch_x4s.clear()

        # periodic checkpoint
        if i % 500 == 0 and i > 0 and (time.time() - last_checkpoint) > 300:
            ckpt_path = os.path.join(out_dir, "bags_partial.json")
            with open(ckpt_path, "w") as f:
                json.dump(bags, f)
            print(f"[CKPT] Saved partial checkpoint ({len(bags)} imgs) → {ckpt_path}")
            last_checkpoint = time.time()

    return bags


In [ ]:
def discover_image_dirs(base_dir: str, splits: Optional[List[str]]) -> List[str]:
    candidates = []
    if splits is None:
        for root, dirs, files in os.walk(base_dir):
            if any(f.endswith("_green.png") for f in files):
                candidates.append(root)
    else:
        for sp in splits:
            p = os.path.join(base_dir, sp)
            if os.path.isdir(p):
                candidates.append(p)
    return candidates

In [ ]:
def main():
    import argparse, subprocess, time, shutil

    parser = argparse.ArgumentParser()
    parser.add_argument("--chunk-index", type=int, default=0,
                        help="0-based chunk index: 0,1,2,...")
    parser.add_argument("--chunk-size", type=int, default=500,
                        help="number of images per chunk")
    parser.add_argument("--max-hours", type=float, default=7.0,
                        help="optional wall-clock safety stop")
    parser.add_argument("--kaggle-dataset", type=str, default="",
                        help="username/dataset-name for Kaggle upload (optional)")
    args, _ = parser.parse_known_args()

    cfg = Cfg()
    print("Base dir:", cfg.base_dir)
    print("Out  dir:", cfg.out_dir)
    ensure_dir(cfg.out_dir)

    if models is None:
        print("[ERROR] Cellpose not available. Run: !pip install -q 'cellpose[all]>=3.1' tifffile scikit-image")
        return

    start_time = time.time()

    # -------------------------------------------------------------
    # Discover all training stems once
    stems_all = sorted([
        os.path.basename(p).replace("_green.png", "")
        for p in glob.glob(os.path.join(cfg.base_dir, "train", "*_green.png"))
    ])
    total = len(stems_all)
    print(f"[INFO] Total images in train: {total}")

    # Explicit chunk slicing (no resume-by-masks magic)
    start = args.chunk_index * args.chunk_size
    end   = min(start + args.chunk_size, total)
    stems_subset = stems_all[start:end]

    print(f"[CHUNK] chunk-index={args.chunk_index}")
    print(f"[CHUNK] processing images {start}–{end-1} (count={len(stems_subset)})")
    if not stems_subset:
        print("[INFO] Nothing to process for this chunk (check chunk-index / size).")
        return

    # -------------------------------------------------------------
    # Process only the chosen stems in "train"
    d = os.path.join(cfg.base_dir, "train")
    out_d = os.path.join(cfg.out_dir, os.path.basename(d))
    ensure_dir(out_d)

    full_bags: Dict[str, List[Dict]] = {}

    bags = process_folder(d, out_d, cfg, stems_override=stems_subset)
    for stem, lst in bags.items():
        full_bags[f"{os.path.basename(d)}::{stem}"] = lst

    # Safety stop checks (disk/time) – optional here, but you can keep them:
    free_gb = shutil.disk_usage("/kaggle/working").free / 1e9
    elapsed_hr = (time.time() - start_time) / 3600
    print(f"[INFO] Free space: {free_gb:.2f} GB, elapsed time: {elapsed_hr:.2f} h")

    # -------------------------------------------------------------
    # Save results for THIS chunk
    chunk_num = args.chunk_index + 1  # 1-based for filenames
    bags_path = os.path.join(cfg.out_dir, f"bags_chunk{chunk_num}.json")
    with open(bags_path, "w") as f:
        json.dump(full_bags, f, indent=2)
    print(f"[SAVE] Bags mapping: {bags_path}")

    readme = os.path.join(cfg.out_dir, "README.txt")
    with open(readme, "w") as f:
        f.write(
            f"""Outputs
=======
- *_mask.png : cell instance mask (16-bit labels)
- *_overlay.png : QC overlay (optional)
- <stem>.npy : normalized 4-channel image
- bags_chunk{chunk_num}.json : cell metadata for this chunk

Chunk info
----------
- chunk-index (0-based): {args.chunk_index}
- chunk-size: {args.chunk_size}
- image range: [{start}, {end})
"""
        )
    print(f"[SAVE] README: {readme}")

    # -------------------------------------------------------------
    # Zip the results of current out_dir
    zip_path = f"/kaggle/working/hpa_seg_chunk{chunk_num}.zip"
    print(f"[ZIP] Compressing results to {zip_path} ...")
    try:
        shutil.make_archive(zip_path.replace(".zip", ""), "zip", cfg.out_dir)
        print(f"[ZIP] Done. Size = {os.path.getsize(zip_path)/1e9:.2f} GB")
    except Exception as e:
        print(f"[WARN] Zipping failed: {e}")

    # Optional Kaggle dataset upload
    if args.kaggle_dataset:
        try:
            subprocess.run([
                "kaggle", "datasets", "version",
                "-p", os.path.dirname(zip_path),
                "-m", f"HPA segmentation chunk {chunk_num}",
                "--dir-mode", "zip",
                "--quiet"
            ], check=True)
            print("[KAGGLE] Upload successful ✅")
        except Exception as e:
            print("[KAGGLE] Upload failed ❌:", e)

    print(f"[DONE] Chunk {chunk_num} complete.")


In [ ]:
import sys

sys.argv = [
    "segmentation",
    "--chunk-index", "6",
    "--chunk-size",  "500",
]
main()


Base dir: /kaggle/input/hpa-single-cell-image-classification
Out  dir: /kaggle/working/hpa_segmentation
[INFO] Total images in train: 21806
[CHUNK] chunk-index=6
[CHUNK] processing images 3000–3499 (count=500)
[INFO] Using provided subset: 500 stems


100%|██████████| 1.15G/1.15G [00:04<00:00, 300MB/s]
Segmenting train:   1%|▏         | 7/500 [00:05<06:46,  1.21it/s]

[SEG] 23722a9e-bbab-11e8-b2ba-ac1f6b6435d0: raw=42, kept=42
[SEG] 237554c2-bbb9-11e8-b2ba-ac1f6b6435d0: raw=41, kept=36
[SEG] 2378c180-bb9a-11e8-b2b9-ac1f6b6435d0: raw=697, kept=50
[SEG] 237c6484-bbb3-11e8-b2ba-ac1f6b6435d0: raw=24, kept=22
[SEG] 23837d58-bbba-11e8-b2ba-ac1f6b6435d0: raw=47, kept=37
[SEG] 23854174-bbc9-11e8-b2bc-ac1f6b6435d0: raw=34, kept=30
[SEG] 2387d3e6-bb9f-11e8-b2b9-ac1f6b6435d0: raw=26, kept=26
[SEG] 238fe72e-bba5-11e8-b2ba-ac1f6b6435d0: raw=71, kept=49


Segmenting train:   3%|▎         | 15/500 [03:22<44:22,  5.49s/it]  

[SEG] 239b49f8-bbb3-11e8-b2ba-ac1f6b6435d0: raw=61, kept=18
[SEG] 239bf182-bbc2-11e8-b2bb-ac1f6b6435d0: raw=54, kept=52
[SEG] 239d75da-bbbc-11e8-b2ba-ac1f6b6435d0: raw=20, kept=19
[SEG] 239d7716-bbac-11e8-b2ba-ac1f6b6435d0: raw=1924, kept=56
[SEG] 239f0bc4-bbb3-11e8-b2ba-ac1f6b6435d0: raw=7, kept=6
[SEG] 23a0fae4-bb9d-11e8-b2b9-ac1f6b6435d0: raw=337, kept=53
[SEG] 23a1b34c-bba4-11e8-b2b9-ac1f6b6435d0: raw=251, kept=54
[SEG] 23a4ec7a-bbb2-11e8-b2ba-ac1f6b6435d0: raw=22, kept=22


...

Segmenting train:  54%|█████▍    | 272/500 [2:02:49<5:04:15, 80.07s/it]

[SEG] 26a4590c-bbac-11e8-b2ba-ac1f6b6435d0: raw=5, kept=4


Segmenting train:  56%|█████▌    | 279/500 [2:02:55<26:48,  7.28s/it]  

[SEG] 26a4c948-bba0-11e8-b2b9-ac1f6b6435d0: raw=172, kept=74
[SEG] 26aaf878-bba9-11e8-b2ba-ac1f6b6435d0: raw=41, kept=38
[SEG] 26ab618a-bb9c-11e8-b2b9-ac1f6b6435d0: raw=144, kept=119
[SEG] 26bd20ba-bba0-11e8-b2b9-ac1f6b6435d0: raw=310, kept=34
[SEG] 26bd8418-bbb6-11e8-b2ba-ac1f6b6435d0: raw=130, kept=125
[SEG] 26be893e-bba8-11e8-b2ba-ac1f6b6435d0: raw=46, kept=19
[SEG] 26c3cd14-bbb1-11e8-b2ba-ac1f6b6435d0: raw=28, kept=26
[SEG] 26c47c0a-bba7-11e8-b2ba-ac1f6b6435d0: raw=47, kept=45


Segmenting train:  57%|█████▋    | 287/500 [2:07:05<25:01,  7.05s/it]  

In [ ]:
import glob, os

root = "/kaggle/working/hpa_segmentation"  # adjust if needed
npy_files   = glob.glob(os.path.join(root, "**", "*.npy"), recursive=True)
mask_files  = glob.glob(os.path.join(root, "**", "*_mask.png"), recursive=True)

print("Num .npy images:", len(npy_files))
print("Num mask images:", len(mask_files))
print("First 3 npy:", npy_files[:3])
print("First 3 masks:", mask_files[:3])


Num .npy images: 500
Num mask images: 500
First 3 npy: ['/kaggle/working/hpa_segmentation/train/22dbb108-bba7-11e8-b2ba-ac1f6b6435d0.npy', '/kaggle/working/hpa_segmentation/train/1f867b02-bb9b-11e8-b2b9-ac1f6b6435d0.npy', '/kaggle/working/hpa_segmentation/train/1f6c7d26-bba3-11e8-b2b9-ac1f6b6435d0.npy']
First 3 masks: ['/kaggle/working/hpa_segmentation/train/2189fa42-bbc7-11e8-b2bc-ac1f6b6435d0_mask.png', '/kaggle/working/hpa_segmentation/train/228d86aa-bba1-11e8-b2b9-ac1f6b6435d0_mask.png', '/kaggle/working/hpa_segmentation/train/1e4f890a-bbb8-11e8-b2ba-ac1f6b6435d0_mask.png']


**Do not run this before downloading the zip file.**

In [ ]:
!rm -rf /kaggle/working/hpa_segmentation
!rm -rf /kaggle/working/hpa_seg_chunk6.zip

In [ ]:
!zip -rq /kaggle/working/hpa_seg_chunk4.zip /kaggle/working/hpa_segmentation \

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!pip install -q PyDrive2

from google.colab import auth
auth.authenticate_user()

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

print("✅ Authenticated with Google Drive")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 85.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.2 MB/s eta 0:00:00
✅ Authenticated with Google Drive


In [ ]:
import glob, os

# Your target Drive folder
folder_id = "1u-wOS7X62Qhu13px42X-SLoymRAfYFdD"

# Location of your ZIPs inside the notebook environment
# If you're in Colab and copied from Kaggle, adapt this:
zip_files = glob.glob("/kaggle/working/hpa_seg_chunk*.zip")  # or "/content/hpa_seg_chunk*.zip"

print(f"Found {len(zip_files)} ZIP files:", zip_files)

for path in zip_files:
    name = os.path.basename(path)
    print(f"📤 Uploading {name} ...")
    file_drive = drive.CreateFile({
        "title": name,
        "parents": [{"id": folder_id}],
    })
    file_drive.SetContentFile(path)

    try:
        file_drive.Upload()
        print(f"✅ Uploaded '{name}' → https://drive.google.com/file/d/{file_drive['id']}/view\n")
    except Exception as e:
        # This will show the real HTTP error instead of the JSONDecodeError
        print(f"❌ Upload failed for {name}: {e}")
        raise


Found 1 ZIP files: ['/kaggle/working/hpa_seg_chunk7.zip']
📤 Uploading hpa_seg_chunk7.zip ...
❌ Upload failed for hpa_seg_chunk7.zip: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)


InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)